In [ ]:
import requests, json, re
import ipywidgets as widgets

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
exec_params = {
    'output_format' : 'json'
}

In [ ]:
lang_opts=['Arabic','Hebrew','Sabaic','Ugaritic','Aramaic','Amharic','Phoenician','Syriac','Geez','Akkadian','Qatabanic','Eblaite']
sem_langs=widgets.SelectMultiple(
    options=lang_opts,
    value=[],
    description='Languages',
    disabled=False
)

In [ ]:
sem_langs

In [ ]:
word_roots = []

In [ ]:
for num in range(1,468):
    try:
        resp=requests.get("http://www.semiticroots.net/index.php?r=root/view&id={0}".format(num))
        chk_num=0
        all_pres=False
        tab0 = BeautifulSoup(resp.text, 'lxml').find_all(class_="detail-view")[0]
        c_root = BeautifulSoup(resp.text, 'lxml').find_all('h1')[0].text.split(':')[1]
        derivs=tab0.text.split('\n')[-2].replace('Derivatives','')
        cncpt=tab0.text.split('\n')[-3].replace('Concept','')
        for lng in sem_langs.value:
            if lng.lower() in derivs.lower(): 
                chk_num+=1
        if chk_num == len(sem_langs.value)-1:
            word_roots.append(str({
                "root" : c_root,
                "concept" : cncpt,
                "derivatives": derivs
            }))
    except Exception as e:
        print(e)
        pass

In [ ]:
def write_roots(word_roots = [], out_form='json', sem_langs=[]):
    if out_form.lower() == 'json':
        with open("outputs/common-roots_{0}.json".format("-".join(sem_langs)),'w') as fh:
            fh.writelines(word_roots)
            print("file written: outputs/common-roots_{0}.json".format("-".join(sem_langs)))
        return
    if out_form.lower() == 'csv':
        root_dicts = []
        for rt in rtslist:
            try:
                root_dicts.append(root_dicts.loads(rt))
            except:
                print(rt)
        csv_data=[]

        for x in root_dicts:
            try:
                for lang in lang_opts:
                    x['derivatives'] = x['derivatives'].replace(lang," {0}:".format(lang))
                csv_data.append("{0},{1},{2}".format(x['concept'],x['root'],x['derivatives']))
                with open("outputs/common-roots_{0}.csv".format("-".join(sem_langs)),'w') as rtfile:
                    rtfile.write("\n".join(sorted(csv_data)))
                print("file written: outputs/common-roots_{0}.csv".format("-".join(sem_langs)))
            except:
                print(type(x))
    

In [ ]:
write_roots(word_roots,exec_params['output_format'],sem_langs.value)